In [4]:
import torch
from TTS.api import TTS


In [7]:

# Get device
device = "cuda" if torch.cuda.is_available() else "cpu"

# List available 🐸TTS models
print(TTS().list_models())

# Init TTS
tts = TTS("tts_models/multilingual/multi-dataset/xtts_v2").to(device)


 > You must confirm the following:
 | > "I have purchased a commercial license from Coqui: licensing@coqui.ai"
 | > "Otherwise, I agree to the terms of the non-commercial CPML: https://coqui.ai/cpml" - [y/n]
 > Downloading model to C:\Users\Admin\AppData\Local\tts\tts_models--multilingual--multi-dataset--xtts_v2


100%|██████████| 1.87G/1.87G [00:35<00:00, 53.0MiB/s]
100%|██████████| 4.37k/4.37k [00:00<00:00, 9.10kiB/s]
100%|██████████| 361k/361k [00:00<00:00, 543kiB/s] 
100%|██████████| 32.0/32.0 [00:00<00:00, 44.9iB/s]
 94%|█████████▎| 7.26M/7.75M [00:00<00:00, 26.9MiB/s]

 > Model's license - CPML
 > Check https://coqui.ai/cpml.txt for more info.
 > Using model: xtts


100%|██████████| 7.75M/7.75M [00:12<00:00, 26.9MiB/s]

In [8]:

# Run TTS
# ❗ Since this model is multi-lingual voice cloning model, we must set the target speaker_wav and language
# Text to speech list of amplitude values as output
wav = tts.tts(text="장발장, 당신은 더 이상 악한 사람이 아닙니다. 나는 당신의 영혼을 샀습니다. 나는 그것을 하나님께 바쳤습니다. 이제 당신은 선한 사람이 되어야 합니다.",
               speaker_wav="mp3_raw/미리엘_클로버.mp3", language="ko")
# Text to speech to a file


 > Text splitted to sentences.
['장발장, 당신은 더 이상 악한 사람이 아닙니다.', '나는 당신의 영혼을 샀습니다.', '나는 그것을 하나님께 바쳤습니다.', '이제 당신은 선한 사람이 되어야 합니다.']


The attention mask is not set and cannot be inferred from input because pad token is same as eos token.As a consequence, you may observe unexpected behavior. Please pass your input's `attention_mask` to obtain reliable results.
c:\Users\Admin\miniconda3\envs\gumi_env310\lib\site-packages\transformers\models\gpt2\modeling_gpt2.py:544: UserWarning: 1Torch was not compiled with flash attention. (Triggered internally at ..\aten\src\ATen\native\transformers\cuda\sdp_utils.cpp:455.)
  attn_output = torch.nn.functional.scaled_dot_product_attention(


 > Processing time: 15.269226312637329
 > Real-time factor: 0.5118931232875187


In [11]:
tts.tts_to_file(text="이게 누구시죠, 저를 쳐다보고 있는 이 선량한 사람은?",
                 speaker_wav="mp3_raw/미리엘_클로버.mp3", language="ko", file_path="mp3_make/output.wav")

 > Text splitted to sentences.
['이게 누구시죠, 저를 쳐다보고 있는 이 선량한 사람은?']
 > Processing time: 3.1566522121429443
 > Real-time factor: 0.43084691788248936


'mp3_make/output.wav'

In [22]:
import gradio as gr

# 사용자 입력을 처리하는 함수
def process_input(text, audio):
    # 텍스트와 오디오 파일을 채팅창에 출력
    return [(text, None), (None, gr.Audio(audio))]

# Gradio 인터페이스 구성
with gr.Blocks() as demo:
    gr.Markdown("## 구미챗봇")
    
    # 챗봇 출력창
    chatbot = gr.Chatbot(label="구미챗봇")
    
    # 텍스트 입력 박스
    text_input = gr.Textbox(label="사용자 메시지 입력")
    
    # 음성 파일 입력 박스
    audio_input = gr.Audio(sources="upload", type="filepath", label="음성 파일을 업로드하세요")
    
    # 입력을 제출하는 버튼
    submit_btn = gr.Button("제출")
    
    # 버튼 클릭 시 process_input 함수 호출
    submit_btn.click(process_input, inputs=[text_input, audio_input], outputs=chatbot)

# 인터페이스 실행
demo.launch()


Running on local URL:  http://127.0.0.1:7880

To create a public link, set `share=True` in `launch()`.


In [34]:
import gradio as gr

# 텍스트와 음성 파일을 처리하는 함수
def chatbot_send_message(message):
    responses = []
    system_message = "audio_file:"
    responses.append[None,system_message]
    responses.append([message,None])
    # 특정 경로에 있는 음성 파일 읽기
    specific_audio_path = "mp3_make/output.wav"  # 여기에 원하는 파일 경로를 입력하세요.
    if specific_audio_path:
        # 특정 경로의 음성 파일을 Gradio Audio 컴포넌트로 출력
        audio_component = gr.Audio(specific_audio_path)  # 특정 경로에 있는 파일을 읽어들임

        responses.append([None, audio_component])
    print(responses)
    # 채팅 메시지와 입력창을 초기화하여 반환
    return responses, ""

with gr.Blocks() as app:
    with gr.Tab("음성 인식봇"):
        with gr.Column():
            gr.Markdown(
                value="""
<center><h1> 음성 인식 봇 </h1></center>
<center> AI 인공지능 봇입니다. 음성으로 묻거나 문서 요약, 일정 관리 등을 할 수 있습니다.</center>
"""
            )
            cb_chatbot = gr.Chatbot(
                value=[[None, "안녕하세요 AI 챗봇입니다. 무엇이든 질문하세요"]],
                label="구미챗봇",
            )
        with gr.Row():
            cb_textbox = gr.Textbox(
                lines=3,
                placeholder="입력창입니다.",
                container=False,
                scale=5
            ) # 채팅 입력

            audio_input = gr.Audio(
                sources="microphone",
                type="filepath",
                label="음성을 입력해주세요",
                scale=2
            ) # 음성 입력

            cb_send_btn = gr.Button(
                value="보내기",
                visible="primary",
                scale=1
            ) # 실행 버튼
            
            file_upload = gr.File(
                label="txt 파일을 업로드해주세요",
                type="filepath"
            ) # 파일 업로드
            
        with gr.Row():
            gr.Button(
                value="되돌리기"
            )
            gr.Button(
                value="초기화"
            )
        
        # 텍스트 입력 처리
        cb_send_btn.click(
            fn=chatbot_send_message,
            inputs=[cb_textbox],
            outputs=[cb_chatbot, cb_textbox]
        )
        
        # 파일 업로드 처리
        file_upload.change(
            fn=chatbot_send_message,
            inputs=[file_upload],
            outputs=[cb_chatbot]
        )

    with gr.Tab("문서 요약봇"):
        pass
    with gr.Tab("일정 관리봇"):
        pass

app.launch(debug=True)


Running on local URL:  http://127.0.0.1:7881

To create a public link, set `share=True` in `launch()`.


Traceback (most recent call last):
  File "c:\Users\Admin\miniconda3\envs\gumi_env310\lib\site-packages\gradio\queueing.py", line 536, in process_events
    response = await route_utils.call_process_api(
  File "c:\Users\Admin\miniconda3\envs\gumi_env310\lib\site-packages\gradio\route_utils.py", line 288, in call_process_api
    output = await app.get_blocks().process_api(
  File "c:\Users\Admin\miniconda3\envs\gumi_env310\lib\site-packages\gradio\blocks.py", line 1931, in process_api
    result = await self.call_function(
  File "c:\Users\Admin\miniconda3\envs\gumi_env310\lib\site-packages\gradio\blocks.py", line 1516, in call_function
    prediction = await anyio.to_thread.run_sync(  # type: ignore
  File "c:\Users\Admin\miniconda3\envs\gumi_env310\lib\site-packages\anyio\to_thread.py", line 56, in run_sync
    return await get_async_backend().run_sync_in_worker_thread(
  File "c:\Users\Admin\miniconda3\envs\gumi_env310\lib\site-packages\anyio\_backends\_asyncio.py", line 2177, in ru

Keyboard interruption in main thread... closing server.


In [35]:
import gradio as gr

# 텍스트와 음성 파일을 처리하는 함수
def chatbot_send_message(message):
    responses = []
    
    # 사용자 입력 메시지 추가
    responses.append([message, None])
    
    # 시스템 메시지 추가
    system_message = "audio_file:"
    responses.append([None, system_message])
    
    # 특정 경로에 있는 음성 파일 읽기
    specific_audio_path = "mp3_make/output.wav"  # 여기에 원하는 파일 경로를 입력하세요.
    if specific_audio_path:
        # 특정 경로의 음성 파일을 Gradio Audio 컴포넌트로 출력
        audio_component = gr.Audio(specific_audio_path)  # 특정 경로에 있는 파일을 읽어들임
        responses.append([None, audio_component])
    
    print(responses)
    # 채팅 메시지와 입력창을 초기화하여 반환
    return responses, ""

with gr.Blocks() as app:
    with gr.Tab("음성 인식봇"):
        with gr.Column():
            gr.Markdown(
                value="""
<center><h1> 음성 인식 봇 </h1></center>
<center> AI 인공지능 봇입니다. 음성으로 묻거나 문서 요약, 일정 관리 등을 할 수 있습니다.</center>
"""
            )
            cb_chatbot = gr.Chatbot(
                value=[[None, "안녕하세요 AI 챗봇입니다. 무엇이든 질문하세요"]],
                label="구미챗봇",
            )
        with gr.Row():
            cb_textbox = gr.Textbox(
                lines=3,
                placeholder="입력창입니다.",
                container=False,
                scale=5
            ) # 채팅 입력

            cb_send_btn = gr.Button(
                value="보내기",
                visible="primary",
                scale=1
            ) # 실행 버튼
            
            file_upload = gr.File(
                label="txt 파일을 업로드해주세요",
                type="filepath"
            ) # 파일 업로드
            
        with gr.Row():
            gr.Button(
                value="되돌리기"
            )
            gr.Button(
                value="초기화"
            )
        
        # 텍스트 입력 처리
        cb_send_btn.click(
            fn=chatbot_send_message,
            inputs=[cb_textbox],
            outputs=[cb_chatbot, cb_textbox]
        )
        
        # 파일 업로드 처리
        file_upload.change(
            fn=chatbot_send_message,
            inputs=[file_upload],
            outputs=[cb_chatbot]
        )

    with gr.Tab("문서 요약봇"):
        pass
    with gr.Tab("일정 관리봇"):
        pass

app.launch(debug=True)


Running on local URL:  http://127.0.0.1:7881

To create a public link, set `share=True` in `launch()`.


[['ddd', None], [None, 'audio_file:'], [None, <gradio.components.audio.Audio object at 0x00000269F6D5F8E0>]]
Keyboard interruption in main thread... closing server.


In [36]:
import gradio as gr

# 텍스트와 음성 파일을 처리하는 함수
def chatbot_send_message(message):
    responses = []
    
    # 사용자 입력 메시지 추가 (튜플 형태)
    responses.append((message, None))
    
    # 시스템 메시지 추가 (튜플 형태)
    system_message = "audio_file:"
    responses.append((None, system_message))
    
    # 특정 경로에 있는 음성 파일 읽기
    specific_audio_path = "mp3_make/output.wav"  # 여기에 원하는 파일 경로를 입력하세요.
    if specific_audio_path:
        # 특정 경로의 음성 파일을 Gradio Audio 컴포넌트로 출력 (튜플 형태)
        audio_component = gr.Audio(specific_audio_path)  # 특정 경로에 있는 파일을 읽어들임
        responses.append((None, audio_component))
    
    print(responses)
    # 채팅 메시지와 입력창을 초기화하여 반환 (튜플 형태)
    return responses, ""

with gr.Blocks() as app:
    with gr.Tab("음성 인식봇"):
        with gr.Column():
            gr.Markdown(
                """
<center><h1> 음성 인식 봇 </h1></center>
<center> AI 인공지능 봇입니다. 음성으로 묻거나 문서 요약, 일정 관리 등을 할 수 있습니다.</center>
"""
            )
            cb_chatbot = gr.Chatbot(
                value=[(None, "안녕하세요 AI 챗봇입니다. 무엇이든 질문하세요")],
                label="구미챗봇",
            )
        with gr.Row():
            cb_textbox = gr.Textbox(
                lines=3,
                placeholder="입력창입니다.",
                container=False,
                scale=5
            )  # 채팅 입력

            cb_send_btn = gr.Button(
                value="보내기",
                visible="primary",
                scale=1
            )  # 실행 버튼
            
            file_upload = gr.File(
                label="txt 파일을 업로드해주세요",
                type="filepath"
            )  # 파일 업로드
            
        with gr.Row():
            gr.Button(
                value="되돌리기"
            )
            gr.Button(
                value="초기화"
            )
        
        # 텍스트 입력 처리
        cb_send_btn.click(
            fn=chatbot_send_message,
            inputs=[cb_textbox],
            outputs=[cb_chatbot, cb_textbox]
        )
        
        # 파일 업로드 처리
        file_upload.change(
            fn=chatbot_send_message,
            inputs=[file_upload],
            outputs=[cb_chatbot]
        )

    with gr.Tab("문서 요약봇"):
        pass
    with gr.Tab("일정 관리봇"):
        pass

app.launch(debug=True)


Running on local URL:  http://127.0.0.1:7881

To create a public link, set `share=True` in `launch()`.


[('dddd', None), (None, 'audio_file:'), (None, <gradio.components.audio.Audio object at 0x00000269F711D900>)]
Keyboard interruption in main thread... closing server.
